In [2]:
from typing import List
import pydicom
from pydicom.dataset import Dataset
from io import BytesIO
import os 
import nibabel as nib
import numpy as np
import torch
from rt_utils import RTStructBuilder

In [5]:
rtstruct_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/"

In [7]:
def load_dicom_datasets(dicom_folder_path: str) -> List[Dataset]:
    """
    Charge tous les fichiers DICOM d'un dossier donné en datasets pydicom.
    """
    dicom_datasets = []
    for filename in os.listdir(dicom_folder_path):
        if filename.endswith('.dcm'):
            file_path = os.path.join(dicom_folder_path, filename)
            try:
                ds = pydicom.dcmread(file_path)
                dicom_datasets.append(ds)
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier DICOM {filename}: {e}")
                continue  # ou lever une exception selon les besoins de votre application

    if not dicom_datasets:
        raise Exception("Aucun fichier DICOM valide trouvé dans le dossier spécifié.")

    return dicom_datasets

rtstruct_file = load_dicom_datasets(rtstruct_path)
print(rtstruct_file[0])

rtstruct = rtstruct_file[0]

In [9]:
print(type(rtstruct))

# Récupérer les diamètres et les volumes des rois

In [22]:
import os
import pydicom
import numpy as np
from shapely.geometry import Polygon
from dicompylercore import dicomparser

# Charger le fichier DICOM RTStruct
rtstruct_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/RS1.2.752.243.1.1.20230602094342956.1800.51115.dcm"
dicoms_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/RM/IRM"

# Charger les images de la série DICOM
dicom_files = [os.path.join(dicoms_path, f) for f in os.listdir(dicoms_path) if f.endswith('.dcm')]
dicom_series = [pydicom.dcmread(f) for f in dicom_files]
dicom_series.sort(key=lambda x: int(x.InstanceNumber))

# Extraire les informations de PixelSpacing et SliceThickness
pixel_spacing = dicom_series[0].PixelSpacing
slice_thickness = dicom_series[0].SliceThickness

# Charger le fichier RTStruct avec dicompyler-core
rtstruct = dicomparser.DicomParser(rtstruct_path)

# Extraire les structures
structures = rtstruct.GetStructures()

# Fonction pour calculer le diamètre d'un ROI
def calculate_diameter(contour):
    max_distance = 0
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            distance = np.linalg.norm(np.array(contour[i]) - np.array(contour[j]))
            if distance > max_distance:
                max_distance = distance
    return max_distance

# Fonction pour calculer le volume d'un ROI
def calculate_volume(coords, thickness):
    volume = 0
    for z in coords.keys():
        contours = coords[z]
        for contour in contours:
            polygon = Polygon(contour['data'])
            volume += polygon.area * thickness
    return volume / 1000  # Convertir en cm³

# Parcourir les structures et extraire les informations
for roi_number, roi_data in structures.items():
    roi_name = roi_data['name']
    print(f"ROI: {roi_name}")
    
    # Obtenir les coordonnées des contours
    coords = rtstruct.GetStructureCoordinates(roi_number)
    
    # Calculer le volume
    thickness = dicomparser.DicomParser.CalculatePlaneThickness(rtstruct, coords)
    volume = calculate_volume(coords, thickness)
    
    # Calculer le diamètre maximal
    contours = []
    for plane in coords.values():
        for contour in plane:
            contours.append(contour['data'])
    diameters = [calculate_diameter(contour) for contour in contours]
    
    # Obtenir les slices de début et de fin
    start_slices = [contour[0][2] for contour in contours if contour]
    end_slices = [contour[-1][2] for contour in contours if contour]

    print(f"  Diamètre max: {max(diameters):.2f} mm")
    print(f"  Volume: {volume:.2f} cm³")
    print(f"  Slice de début: {min(start_slices)}")
    print(f"  Slice de fin: {max(end_slices)}")

# Pipeline fonctionnel OMG

In [28]:
import os
import pydicom
import numpy as np
from shapely.geometry import Polygon
from dicompylercore import dicomparser

# Charger le fichier DICOM RTStruct
rtstruct_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/RS1.2.752.243.1.1.20230602094342956.1800.51115.dcm"
dicoms_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/RM/IRM"

# Charger les images de la série DICOM
dicom_files = [os.path.join(dicoms_path, f) for f in os.listdir(dicoms_path) if f.endswith('.dcm')]
dicom_series = [pydicom.dcmread(f) for f in dicom_files]
dicom_series.sort(key=lambda x: int(x.InstanceNumber))

# Extraire les informations de PixelSpacing et SliceThickness
pixel_spacing = dicom_series[0].PixelSpacing
slice_thickness = dicom_series[0].SliceThickness

# Créer une map des positions de slices à leurs indices
slice_positions = {round(dcm.ImagePositionPatient[2], 2): i+1 for i, dcm in enumerate(dicom_series)}

# Charger le fichier RTStruct avec dicompyler-core
rtstruct = dicomparser.DicomParser(rtstruct_path)

# Extraire les structures
structures = rtstruct.GetStructures()

# Fonction pour calculer le diamètre d'un ROI
def calculate_diameter(contour):
    max_distance = 0
    for i in range(len(contour)):
        for j in range(i + 1, len(contour)):
            distance = np.linalg.norm(np.array(contour[i]) - np.array(contour[j]))
            if distance > max_distance:
                max_distance = distance
    return max_distance

# Fonction pour calculer le volume d'un ROI
def calculate_volume(coords, thickness):
    volume = 0
    for z in coords.keys():
        contours = coords[z]
        for contour in contours:
            polygon = Polygon(contour['data'])
            volume += polygon.area * thickness
    return volume / 1000  # Convertir en cm³

# Parcourir les structures et extraire les informations
for roi_number, roi_data in structures.items():
    roi_name = roi_data['name']
    print(f"ROI: {roi_name}")
    
    # Obtenir les coordonnées des contours
    coords = rtstruct.GetStructureCoordinates(roi_number)
    
    # Calculer le volume
    thickness = dicomparser.DicomParser.CalculatePlaneThickness(rtstruct, coords)
    volume = calculate_volume(coords, thickness)
    
    # Calculer le diamètre maximal
    contours = []
    contour_slice_indices = []
    for plane in coords.values():
        for contour in plane:
            contours.append(contour['data'])
            z_pos = round(contour['data'][0][2], 2)
            if z_pos in slice_positions:
                contour_slice_indices.append(slice_positions[z_pos])
    
    diameters = [calculate_diameter(contour) for contour in contours]
    nb_dicoms = len(dicom_series)
    # Obtenir les slices de début et de fin
    start_slice = nb_dicoms - max(contour_slice_indices) if contour_slice_indices else None
    end_slice = nb_dicoms - min(contour_slice_indices) if contour_slice_indices else None
    
    print(f"  Diamètre max: {max(diameters):.2f} mm")
    print(f"  Volume: {volume:.2f} cm³")
    print(f"  Slice de début: {start_slice}")
    print(f"  Slice de fin: {end_slice}")

In [27]:
print(len(dicom_series))

# Version API (ça marche !!!!)

In [43]:
import os
import pydicom
import numpy as np
from shapely.geometry import Polygon
from dicompylercore import dicomparser

def extract_roi_info(rtstruct, dicom_series):

    dicom_series.sort(key=lambda x: int(x.InstanceNumber))
    
    # Extraire les informations de PixelSpacing et SliceThickness
    pixel_spacing = dicom_series[0].PixelSpacing
    slice_thickness = dicom_series[0].SliceThickness

    # Créer une map des positions de slices à leurs indices
    slice_positions = {round(dcm.ImagePositionPatient[2], 2): i+1 for i, dcm in enumerate(dicom_series)}

    # Charger le fichier RTStruct avec dicompyler-core
    rtstruct = dicomparser.DicomParser(rtstruct)

    # Extraire les structures
    structures = rtstruct.GetStructures()

    # Fonction pour calculer le diamètre d'un ROI
    def calculate_diameter(contour):
        max_distance = 0
        for i in range(len(contour)):
            for j in range(i + 1, len(contour)):
                distance = np.linalg.norm(np.array(contour[i]) - np.array(contour[j]))
                if distance > max_distance:
                    max_distance = distance
        return max_distance

    # Fonction pour calculer le volume d'un ROI
    def calculate_volume(coords, thickness):
        volume = 0
        for z in coords.keys():
            contours = coords[z]
            for contour in contours:
                polygon = Polygon(contour['data'])
                volume += polygon.area * thickness
        return volume / 1000  # Convertir en cm³

    # Initialiser le dictionnaire de résultats
    roi_info = {}

    # Parcourir les structures et extraire les informations
    for roi_number, roi_data in structures.items():
        roi_name = roi_data['name']
        
        # Obtenir les coordonnées des contours
        coords = rtstruct.GetStructureCoordinates(roi_number)
        
        # Calculer le volume
        thickness = dicomparser.DicomParser.CalculatePlaneThickness(rtstruct, coords)
        volume = calculate_volume(coords, thickness)
        
        # Calculer le diamètre maximal
        contours = []
        contour_slice_indices = []
        for plane in coords.values():
            for contour in plane:
                contours.append(contour['data'])
                z_pos = round(contour['data'][0][2], 2)
                if z_pos in slice_positions:
                    contour_slice_indices.append(slice_positions[z_pos])
        
        diameters = [calculate_diameter(contour) for contour in contours]
        nb_dicoms = len(dicom_series)
        
        # Obtenir les slices de début et de fin
        start_slice = nb_dicoms - max(contour_slice_indices) if contour_slice_indices else None
        end_slice = nb_dicoms - min(contour_slice_indices) if contour_slice_indices else None

        # Ajouter les informations au dictionnaire
        roi_info[roi_name] = {
            "diameter_max": max(diameters),
            "volume_cm3": volume,
            "start_slice": start_slice,
            "end_slice": end_slice
        }
    
    return roi_info

###########################################################################################################################
# Simulation de rtstruct et dicoms chargés en mémoire en objet pydicom :
def load_dicom_datasets(dicom_folder_path: str) -> List[Dataset]:
    """
    Charge tous les fichiers DICOM d'un dossier donné en datasets pydicom.
    """
    dicom_datasets = []
    for filename in os.listdir(dicom_folder_path):
        if filename.endswith('.dcm'):
            file_path = os.path.join(dicom_folder_path, filename)
            try:
                ds = pydicom.dcmread(file_path)
                dicom_datasets.append(ds)
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier DICOM {filename}: {e}")
                continue  # ou lever une exception selon les besoins de votre application

    if not dicom_datasets:
        raise Exception("Aucun fichier DICOM valide trouvé dans le dossier spécifié.")

    return dicom_datasets
rtstruct_path = "/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/"
dicom_files = load_dicom_datasets(dicoms_path)
rtstruct_files = load_dicom_datasets(rtstruct_path)
rtstruct = rtstruct_files[0]

print("type de rtstruct : ", type(rtstruct))
print("type de dicom_files", type(dicom_files)," ", type(dicom_files[0]))
###########################################################################################################################

roi_info = extract_roi_info(rtstruct, dicom_files)
print(roi_info)